## Description

We need to create a class that should be used to represent bank account <br>
We want the following functionality and characteristics
* accounts are uniquely identified by an account number (aasume it will just passed in the initializer)
* accounts holder have a last and firs name
* accounsts have an asociated preferred time zone offset (e.g 7 for MST)
* balances need to be zero or higher, and shouldn't be directly settable
* but deposits and withdrawals can be made (given suficients funds)
    * if a withdraw is attempted that would result in negative funds, the transaction should be declined
* a monthly interest rate exist and is applicable to all accounts uniformly. There should be a method that can be called to calculate the interest on the current balance using the current interest rate and add it to a balance
* each deposit and withdrawal must generate a confirmation number composed of:
    * The transaction type: D for deposit, W for withdrawal, I for interest deposit, and X for declined (in which case the balance remain unaffected
    * the account number
    * the time the transaction was made, using UTC
    * an incrementing number (that increments accross all accounts and transactions)
    * for (extreme) simplicity assume that the transaction id starts at zero whenever the program starts
    * the confirmation number should be returned from any of the transaction method (deposit, withdraw)
* create a method that, given a confirmation numeber returns
    * the account number, transaction code (D,W, etc) datetime (UTC format), date time (in whatever timezone is specified in the argument, but more human readable) the transacction id
    * make it so, it is a nicely structure object (so can use dotted notation to access those attributes)
    * I proposefully made it so the desired time zone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)
  

For example, we may have an account with: 

* account number: 140568
* preferred time zone: 7 (MST)
* an exisiting balance: 100.0

Suppose the last transaction ID in the system was 123, and the deposit is made for 50.00 on 2019-03-15T14:59:00 UTC on that account.
The new balance should reflect 150 and the confirmation number return should look like this:
D-140568-20190315145900-124

We also want a method that given the confirmation number returns an object with attributes

* result.account_number -> 140568
* result.transaction_code -> D
* result.transaction_id -> 124
* result.time -> 2019-03-15 07:59:00 (MST)
* result.time_utc -> 2019-03-15T14:59:00

Furthemore, if current interest rate is 0.5% and the account's balance is 100, then the result of calling the deposit_interest method, should result in a new transaction and new balance of 1050. Calling this method should also return a confirmation number

In [62]:
from datetime import timedelta
from  datetime import datetime

class TimeZone:
    '''Timezone is a class created to generate a timezone object'''
    def __init__(self,name, offset_hours, offset_minutes):
        
        if(name is None or len(str(name).strip())==0):      #Validating the name is an string
            raise ValueError('TimeZone name can\'t be zero')
        
        self._name = name.strip()                           # As the name is a sting can be saved as argument
        
        if not isinstance(offset_hours,int):                # Validating offset hour is an integer
            raise ValueError('Hours offset must be an integer')
        
        if not isinstance(offset_minutes,int):                # Validating offset minute is an integer
            raise ValueError('minutes offset must be an integer')
        
        if offset_minutes > 59 or offset_minutes < -59:       #validating the mitues was set with a valid interval
            raise ValueError('Minutes offset must be between -59 and 59')
            
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset
    
    @property                     # gettter for offset
    def offset(self):
        return self._offset
    
    @property                     # gettter for name
    def name(self):
        return self._name
    
    def __repr__(self):
        return (f'TimeZone(name={self.name}, offset_hours={self._offset_hours}, offset_minutes={self._offset_minutes})')

In [44]:
def transaction_ids(start_id):
    '''function to generate an iterable di used in the Account class'''
    while True:
        start_id += 1
        yield start_id

In [81]:
class Account:
    
    '''Class Account has as a parameter account number, first name and last name, time zone and initial balance'''
    
    transaction_counter = transaction_ids(100)      # Already created function to generate an consecutive id
    _interest_rate = 0.5
    
    def __init__(self, account_number, first_name, last_name, timezone=None,     #initializing the class
                initial_balance = 0):   
        self._account_number = account_number
        self._first_name = first_name                             #can't be named _first_name cause it will be setted with a validation function
        self._last_name = last_name                               #can't be named _last_name cause it will be setted with a validation function
    
        if timezone is None:                                      # setting a default timezone
            timezone = TimeZone('UTC', 0, 0)
        self._timezone = timezone
        
        self._balance = initial_balance
        
    @property                                                      # shows the account number
    def account_number(self):
        return self._account_number
    
    @property                                                     # shows the first name
    def first_name(self):
        return self._first_name
    
    @first_name.setter                                         #change the first name but previous validation if is allowed
    def first_name(self,value):
        self._first_name = Account.validation_name(value,'First name')
    
    @property                                                  # shows the last name
    def last_name(self):
        return print(self._last_name)
    
    @last_name.setter                                          # set the last name previous validation
    def last_name(self, value):
        self._last_name = Account.validation_name(value,'Last name')
    
    @property                                                 # shows the balance account
    def balance(self):
        return self._balance
        
    @property                                                 # shows the time zone
    def timezone(self):
        return self._timezone
    
    @timezone.setter                                  #set a new timezone validation previous validation otherwise shows an error
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time zone must be a valid TimeZone object')
        self._timezone = value
    
    @classmethod                                     # classmethod that shows the interest class rate
    def get_interes_rate(cls):
        return cls._interest_rate
    
    @classmethod                                     # set the interest rate to the class
    def set_interest_rate(cls, value):
        if not isinstance(value, float):
            raise ValueError('Interest rate must be a real numbre')
        cls._interest_rate = value
        
    def generate_code(self, transaction_code):    #generate code to show the account information
        dtr_str = datetime.now().strftime("%Y%m%d%H%M%S")
        return f'{transaction_code}-{self.account_number}-{dtr_str}-{next(Account.transaction_counter)}'
    
    @staticmethod                                   #classmethod used in some methods to validate if value is a string
    def validation_name(value,parameter):
        if len(str(value).strip()) == 0:
            raise ValueError(f'{parameter} can\'t be empty value')
        return value.strip()
    
    def make_transaction(self):
        return self.generate_code('Dummy')

In [82]:
account2 = Account(account_number="67890", first_name="Jhoon", last_name="Doe", initial_balance=1000)

# Later, trying to set the invalid name will raise an error
try:
    account2.first_name = "Doe "  # Invalid first name
except ValueError as e:
    print(e)  # Output: First name must be a value.

In [83]:
account2.make_transaction()

'Dummy-67890-20241116113909-101'

In [74]:
account2.generate_code()

'D-67890-20241116113101-101'

In [75]:
next(transaction_ids(100))

101